# Machine Learning Project

## Introduction

The goal of the project is to recognize if the image is a face or not.

Images are greyscale 36x36 pixels images.

To reach the goal, we will try to train a convolutional neural network.

In [1]:
from deep_learning_project.load_data import basic_load
from deep_learning_project.net import FirstNeuralNetwork, LinearRegressionNetwork
import torch
import matplotlib.pyplot as plt
from torch import nn
from deep_learning_project.trainers import BaseTrainer
import os
import json
import datetime
from tqdm import tqdm

CURRENT_FOLDER = '.'
MODEL_FOLDERS = os.path.join(CURRENT_FOLDER, 'models')

c:\python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data

Data is separated in 3 datasets.

Train : to train the ML model.

Valid : to valid the ML model.

Test : to test the ML model.

What is the difference between valid and test datasets. The main differencec is when there are used : valid are used inside the training process but test are used when the training is complete. Why use different datasets to do the same thing (test the generalization of model) ? Some do the validation with the test dataset but it is not scientifically correct because it will include a bias on the model. If we train the model until the test dataset error is the lowest, we effectively train the model for the test dataset... This is why we use two different dataset.

In [2]:
device = "cpu"
parallel = False

if torch.cuda.is_available():
    device = "cuda:0"
    if torch.cuda.device_count() > 1:
        parallel = True


valid_size = 0.2
batch_size = 32

print(f"Running on {device}. Parallel= {parallel}.")

data = basic_load(valid_size=valid_size, batch_size=batch_size, device=device)
train_loader = data[0]
valid_loader = data[1]
test_loader = data[2]
classes = data[3]

Running on cuda:0. Parallel= False.


In [4]:
train_features, train_labels = next(iter(data[0]))

print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

img = train_features[0].squeeze() # from 2d to 1d, works only when the data is 1d [[x]] => [x]
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()

print(f"Label: {label}")

ValueError: not enough values to unpack (expected 2, got 1)

In [3]:
# loading to gpu
for i, data in tqdm(enumerate(train_loader, 0)):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

for i, data in tqdm(enumerate(valid_loader, 0)):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

for i, data in tqdm(enumerate(test_loader, 0)):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

2293it [00:24, 93.92it/s]
574it [00:06, 94.67it/s]
239it [00:05, 45.97it/s]


Printing one image of the training set.


## Initializing the FirstNeuralNetwork network

We initialize the network.

Print the configuration.

Then predict a random input.

---

FirstNeuralNetwork is the neural network given by the teacher.

In [5]:
epochs = 50
learning_rate = 0.001

loss_fn = nn.CrossEntropyLoss()


model = LinearRegressionNetwork()
if parallel:
    model = nn.DataParallel(model)
model.to(device)

LinearRegressionNetwork(
  (fc1): Linear(in_features=1296, out_features=2, bias=True)
)

In [6]:
random_shit = torch.rand((1, 1, 36, 36), device=device)

logits = model(random_shit)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1], device='cuda:0')


## Optimizing the network

In [7]:
date_now = datetime.datetime.now()
model_folder_name = os.path.join(MODEL_FOLDERS, date_now.strftime("%Y%m%d_%H%M") + '_' + str(model.__class__.__name__))
os.makedirs(model_folder_name, exist_ok=True)

In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
trainer = BaseTrainer(model, loss_fn, optimizer, checkpoints_path=model_folder_name)

In [9]:
trainer.fit(train_loader=train_loader,
            test_loader=test_loader,
            epochs=epochs,
            device=device)

2293it [00:26, 86.38it/s]


epoch 1 of 50: train_loss: 0.33787, train_accuracy: 67.44%, test_loss: 0.69913, test_accuracy: 67.44%


2293it [00:25, 88.43it/s]


epoch 2 of 50: train_loss: 0.28131, train_accuracy: 70.86%, test_loss: 0.75028, test_accuracy: 65.63%


778it [00:08, 87.70it/s]


KeyboardInterrupt: 

In [ ]:
(best_epoch, best_accuracy, best_loss, best_model) = trainer.get_best_model()

stats_file_data = {
    "device": device,
    "network": str(model.__class__.__name__),
    "epochs_number": epochs,
    "learning_rate": learning_rate,
    "batch_size": batch_size,
    "train_len_data": len(train_loader.dataset), 
    "test_len_data": len(test_loader.dataset),
    "trainer": str(trainer.__class__.__name__),
    "optimizer": str(optimizer.__class__.__name__),
    "pytorch_version": torch.__version__,
    "loss_function": str(loss_fn.__class__.__name__),
    "best_epoch": best_epoch,
    "best_accuracy": best_accuracy,
    "best_loss": best_loss,
    "performances" : trainer.get_stats(),
}

stats_file_data_json = json.dumps(stats_file_data, indent=4)

stat_file_name = os.path.join(model_folder_name, 'stats.json')
model_file_name = os.path.join(model_folder_name, 'weights.pt')
best_model_file_name = os.path.join(model_folder_name, 'best_weights.pt')

with open(os.path.normpath(stat_file_name), 'w', encoding = 'utf-8') as file:
    file.write(stats_file_data_json)

torch.save(model.state_dict(), model_file_name)
torch.save(best_model, best_model_file_name)

In [ ]:
# load model
# ld_model = torch.load('model.pt')

# test_loop(test_loader, ld_model, loss_fn)